In [1]:
# Atm simulator
# Using python and sql

import sqlite3

conn = sqlite3.connect("atm2.db")
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS accounts (
    acc_no INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    pin TEXT,
    balance REAL
)
''')
conn.commit()

def create_account(name, pin, balance):
    cur.execute("INSERT INTO accounts (name, pin, balance) VALUES (?, ?, ?)", (name, pin, balance))
    conn.commit()
    acc_no = cur.lastrowid
    print(f"Account created successfully! Your account number is: {acc_no}")

def login(acc_no, pin):
    cur.execute("SELECT * FROM accounts WHERE acc_no=? AND pin=?", (acc_no, pin))
    user = cur.fetchone()
    if user:
        print(f"Welcome {user[1]}!")
        return user
    else:
        print("Invalid account number or PIN.")
        return None

def check_balance(acc_no):
    cur.execute("SELECT balance FROM accounts WHERE acc_no=?", (acc_no,))
    balance = cur.fetchone()[0]
    print(f"Current Balance: $ {balance}")

def deposit(acc_no, amount):
    cur.execute("UPDATE accounts SET balance = balance + ? WHERE acc_no=?", (amount, acc_no))
    conn.commit()
    print(f"{amount} deposited successfully.")

def withdraw(acc_no, amount):
    cur.execute("SELECT balance FROM accounts WHERE acc_no=?", (acc_no,))
    current_balance = cur.fetchone()[0]

    if amount > current_balance:
        print("Insufficient balance!")
    else:
        cur.execute("UPDATE accounts SET balance = balance - ? WHERE acc_no=?", (amount, acc_no))
        conn.commit()
        print(f"{amount} withdrawn successfully.")

def atm_menu():
    while True:
        print("\n===== Welcome to Python ATM =====")
        print("1. Create Account")
        print("2. Login")
        print("3. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            name = input("Enter your name: ")
            pin = input("Set a 4-digit PIN: ")
            try:
                initial_deposit = float(input("Enter initial deposit amount: $"))
                create_account(name, pin, initial_deposit)
            except ValueError:
                print("Invalid deposit amount.")
        elif choice == '2':
            try:
                acc_no = int(input("Enter Account Number: "))
                pin = input("Enter PIN: ")
            except ValueError:
                print("Account number must be a number.")
                continue

            user = login(acc_no, pin)
            if not user:
                continue

            # Logged in — show transaction menu
            while True:
                print("\n--- Menu ---")
                print("1. Check Balance")
                print("2. Deposit")
                print("3. Withdraw")
                print("4. Logout")
                trans_choice = input("Enter your choice: ")

                if trans_choice == '1':
                    check_balance(acc_no)
                elif trans_choice == '2':
                    try:
                        amt = float(input("Enter amount to deposit: $"))
                        deposit(acc_no, amt)
                    except ValueError:
                        print("Invalid amount.")
                elif trans_choice == '3':
                    try:
                        amt = float(input("Enter amount to withdraw: $"))
                        withdraw(acc_no, amt)
                    except ValueError:
                        print("Invalid amount.")
                elif trans_choice == '4':
                    print("Logging out...")
                    break
                else:
                    print("Invalid choice.")
        elif choice == '3':
            print("Thank you for using the ATM. Goodbye!")
            break
        else:
            print("Invalid choice.")

atm_menu()
conn.close()



===== Welcome to Python ATM =====
1. Create Account
2. Login
3. Exit


Enter your choice:  1
Enter your name:  Sneha Rajbhar
Set a 4-digit PIN:  6732
Enter initial deposit amount: $ 4000


Account created successfully! Your account number is: 1

===== Welcome to Python ATM =====
1. Create Account
2. Login
3. Exit


Enter your choice:  2
Enter Account Number:  1
Enter PIN:  6732


Welcome Sneha Rajbhar!

--- Menu ---
1. Check Balance
2. Deposit
3. Withdraw
4. Logout


...

50.0 withdrawn successfully.

--- Menu ---
1. Check Balance
2. Deposit
3. Withdraw
4. Logout


Enter your choice:  4


Logging out...

===== Welcome to Python ATM =====
1. Create Account
2. Login
3. Exit


Enter your choice:  3


Thank you for using the ATM. Goodbye!


In [2]:
# bill management system

# Connect to the database
conn = sqlite3.connect("bills.db")
cur = conn.cursor()

# Create tables
cur.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    phone TEXT
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS bills (
    bill_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS items (
    item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    bill_id INTEGER,
    item_name TEXT,
    quantity INTEGER,
    price REAL,
    FOREIGN KEY(bill_id) REFERENCES bills(bill_id)
)
''')

conn.commit()

# Function to create a new bill
def create_bill(name, phone):
    cur.execute("INSERT INTO customers (name, phone) VALUES (?, ?)", (name, phone))
    customer_id = cur.lastrowid
    cur.execute("INSERT INTO bills (customer_id) VALUES (?)", (customer_id,))
    bill_id = cur.lastrowid
    conn.commit()
    print(f" Bill created successfully for {name}. Bill ID: {bill_id}")
    return bill_id

# Function to add an item to a bill
def add_item_to_bill(bill_id, item_name, quantity, price):
    cur.execute("INSERT INTO items (bill_id, item_name, quantity, price) VALUES (?, ?, ?, ?)",
                (bill_id, item_name, quantity, price))
    conn.commit()
    print(f" Item '{item_name}' added to Bill ID: {bill_id}")

# Function to view a bill
def view_bill(bill_id):
    cur.execute('''
        SELECT c.name, c.phone, b.date
        FROM bills b
        JOIN customers c ON b.customer_id = c.customer_id
        WHERE b.bill_id = ?
    ''', (bill_id,))
    bill_info = cur.fetchone()

    if not bill_info:
        print(" Bill not found.")
        return

    print(f"\n--- Bill ID: {bill_id} ---")
    print(f"Customer: {bill_info[0]}")
    print(f"Phone: {bill_info[1]}")
    print(f"Date: {bill_info[2]}")
    print("\nItems:")
    print("{} {} {} {}".format("Item", "Qty", "Price", "Total"))

    cur.execute('''
        SELECT item_name, quantity, price FROM items WHERE bill_id = ?
    ''', (bill_id,))
    items = cur.fetchall()

    grand_total = 0
    for item in items:
        name, qty, price = item
        total = qty * price
        grand_total += total
        print("{} {} {} {}".format(name, qty, price, total))

    print(f"\nTotal Amount: ₹{grand_total}")

# Function to view items in a bill with item_id
def view_items_in_bill(bill_id):
    cur.execute('''
        SELECT item_id, item_name, quantity, price FROM items WHERE bill_id = ?
    ''', (bill_id,))
    items = cur.fetchall()

    if not items:
        print(" No items found in this bill.")
        return []

    print("\nItems in Bill:")
    print("{} {} {} {}".format("Item ID", "Item Name", "Qty", "Price"))
    for item in items:
        item_id, name, qty, price = item
        print("{} {} {} {}".format(item_id, name, qty, price))
    return items

# Function to delete an item from bill
def delete_item_from_bill():
    try:
        bill_id = int(input("Enter Bill ID: "))
        items = view_items_in_bill(bill_id)
        if not items:
            return

        item_id = int(input("Enter Item ID to delete: "))
        cur.execute("DELETE FROM items WHERE item_id = ? AND bill_id = ?", (item_id, bill_id))
        conn.commit()
        print(f" Item ID {item_id} deleted successfully from Bill ID {bill_id}.")
    except Exception as e:
        print(" Error deleting item:", e)

# Main menu
def bill_menu():
    while True:
        print("\n===== Bill Management System =====")
        print("1. Create New Bill")
        print("2. Add Item to Bill")
        print("3. View Bill")
        print("4. Delete Item from Bill")
        print("5. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            name = input("Enter customer name: ")
            phone = input("Enter phone number: ")
            create_bill(name, phone)
        elif choice == '2':
            bill_id = int(input("Enter Bill ID: "))
            item_name = input("Enter item name: ")
            qty = int(input("Enter quantity: "))
            price = float(input("Enter price per unit: ₹"))
            add_item_to_bill(bill_id, item_name, qty, price)
        elif choice == '3':
            bill_id = int(input("Enter Bill ID to view: "))
            view_bill(bill_id)
        elif choice == '4':
            delete_item_from_bill()
        elif choice == '5':
            print("Exiting Bill Management System. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

bill_menu()
conn.close()


===== Bill Management System =====
1. Create New Bill
2. Add Item to Bill
3. View Bill
4. Delete Item from Bill
5. Exit


Enter your choice:  1
Enter customer name:  Sneha Rajbhar
Enter phone number:  235847221


 Bill created successfully for Sneha Rajbhar. Bill ID: 1

===== Bill Management System =====
1. Create New Bill
2. Add Item to Bill
3. View Bill
4. Delete Item from Bill
5. Exit


Enter your choice:  2
Enter Bill ID:  1
Enter item name:  Pen
Enter quantity:  10
Enter price per unit: ₹ 10


 Item 'Pen' added to Bill ID: 1

===== Bill Management System =====
1. Create New Bill
2. Add Item to Bill
3. View Bill
4. Delete Item from Bill
5. Exit


...


Items in Bill:
Item ID Item Name            Qty Price
1 Pen 10 10.0


Enter Item ID to delete:  1


 Item ID 1 deleted successfully from Bill ID 1.

===== Bill Management System =====
1. Create New Bill
2. Add Item to Bill
3. View Bill
4. Delete Item from Bill
5. Exit


Enter your choice:  5


Exiting Bill Management System. Goodbye!


In [8]:
# Library Management System 

import sqlite3

# Delete existing database file if it exists — fresh database every run
if os.path.exists("lib.db"):
    os.remove("lib.db")

conn = sqlite3.connect("lib.db")
cur = conn.cursor()

# Drop old tables (optional since DB is fresh)
cur.execute("DROP TABLE IF EXISTS issues")
cur.execute("DROP TABLE IF EXISTS borrowers")

# Create tables
cur.execute('''
CREATE TABLE IF NOT EXISTS borrowers (
    borrower_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT,
    copies INTEGER DEFAULT 1
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS issues (
    issue_id INTEGER PRIMARY KEY AUTOINCREMENT,
    borrower_id INTEGER,
    book_id INTEGER,
    issue_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    return_date INTEGER,  -- store 1 if returned, NULL if not
    FOREIGN KEY(borrower_id) REFERENCES borrowers(borrower_id),
    FOREIGN KEY(book_id) REFERENCES books(book_id)
)
''')

conn.commit()

# Preload NCERT Class 12 books into fresh database
books_to_add = [
    ("Physics Part 1 - NCERT Class 12", "NCERT", 3),
    ("Physics Part 2 - NCERT Class 12", "NCERT", 3),
    ("Chemistry - NCERT Class 12", "NCERT", 3),
    ("Mathematics - NCERT Class 12", "NCERT", 3),
    ("Biology - NCERT Class 12", "NCERT", 2)
]
cur.executemany("INSERT INTO books (title, author, copies) VALUES (?, ?, ?)", books_to_add)
conn.commit()

def issue_book(borrower_name, book_id):
    cur.execute("INSERT INTO borrowers (name) VALUES (?)", (borrower_name,))
    borrower_id = cur.lastrowid

    cur.execute("SELECT copies FROM books WHERE book_id = ?", (book_id,))
    book = cur.fetchone()
    if not book:
        print("Book not found.")
        return
    copies = book[0]

    cur.execute('''SELECT COUNT(*) FROM issues 
                   WHERE book_id = ? AND return_date IS NULL''', (book_id,))
    issued_count = cur.fetchone()[0]

    if issued_count >= copies:
        print("No copies available to issue.")
        return

    cur.execute("INSERT INTO issues (borrower_id, book_id) VALUES (?, ?)", (borrower_id, book_id))
    conn.commit()
    print(f"Book ID {book_id} issued to '{borrower_name}' successfully.")

def return_book(borrower_name, book_id):
    cur.execute("SELECT borrower_id FROM borrowers WHERE name = ? ORDER BY borrower_id DESC LIMIT 1", (borrower_name,))
    borrower = cur.fetchone()
    if not borrower:
        print("Borrower not found.")
        return
    borrower_id = borrower[0]

    cur.execute('''SELECT issue_id FROM issues 
                   WHERE borrower_id = ? AND book_id = ? AND return_date IS NULL''', (borrower_id, book_id))
    issue = cur.fetchone()
    if not issue:
        print("No active issue found for this book and borrower.")
        return

    issue_id = issue[0]
    # Instead of datetime, just mark return_date as 1 (means returned)
    cur.execute("UPDATE issues SET return_date = 1 WHERE issue_id = ?", (issue_id,))
    conn.commit()
    print(f"Book ID {book_id} returned by '{borrower_name}' successfully.")

def list_books():
    cur.execute("SELECT book_id, title, author, copies FROM books")
    books = cur.fetchall()
    print("\nAvailable Books:")
    for book_id, title, author, copies in books:
        cur.execute('''SELECT COUNT(*) FROM issues WHERE book_id = ? AND return_date IS NULL''', (book_id,))
        issued = cur.fetchone()[0]
        available = copies - issued
        print(f"ID: {book_id} | Title: {title} | Author: {author} | Available: {available}")

def library_menu():
    while True:
        print("\n===== Library Management System =====")
        print("1. List Books")
        print("2. Issue Book")
        print("3. Return Book")
        print("4. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            list_books()
        elif choice == '2':
            borrower_name = input("Enter Borrower Name: ")
            try:
                book_id = int(input("Enter Book ID to issue: "))
                issue_book(borrower_name, book_id)
            except ValueError:
                print("Invalid input! Book ID must be a number.")
        elif choice == '3':
            borrower_name = input("Enter Borrower Name: ")
            try:
                book_id = int(input("Enter Book ID to return: "))
                return_book(borrower_name, book_id)
            except ValueError:
                print("Invalid input! Book ID must be a number.")
        elif choice == '4':
            print("Exiting Library Management System. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

library_menu()
conn.close()



===== Library Management System =====
1. List Books
2. Issue Book
3. Return Book
4. Exit


Enter your choice:  1



Available Books:
ID: 1 | Title: Physics Part 1 - NCERT Class 12 | Author: NCERT | Available: 3
ID: 2 | Title: Physics Part 2 - NCERT Class 12 | Author: NCERT | Available: 3
ID: 3 | Title: Chemistry - NCERT Class 12 | Author: NCERT | Available: 3
ID: 4 | Title: Mathematics - NCERT Class 12 | Author: NCERT | Available: 3
ID: 5 | Title: Biology - NCERT Class 12 | Author: NCERT | Available: 2

===== Library Management System =====
1. List Books
2. Issue Book
3. Return Book
4. Exit


Enter your choice:  2
Enter Borrower Name:  Sneha
Enter Book ID to issue:  3


Book ID 3 issued to 'Sneha' successfully.

===== Library Management System =====
1. List Books
2. Issue Book
3. Return Book
4. Exit


Enter your choice:  3
Enter Borrower Name:  Sneha
Enter Book ID to return:  3


Book ID 3 returned by 'Sneha' successfully.

===== Library Management System =====
1. List Books
2. Issue Book
3. Return Book
4. Exit


Enter your choice:  4


Exiting Library Management System. Goodbye!
